In [ ]:
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,plot_confusion_matrix
from sklearn import tree
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
import copy
from sklearn.model_selection import KFold
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

import sys
import random
from sklearn.cluster import KMeans

import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA, FastICA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import FactorAnalysis
from sklearn.mixture import GaussianMixture
from scipy.stats import kurtosis
sns.set(context="notebook", palette="Spectral", style = 'darkgrid' ,font_scale = 1.5, color_codes=True)
colors = ['blue','green','purple','orange','m','y','forestgreen','darkorange','darkblue','indigo','brown','steelblue','chocolate','olivedrab']

import warnings
warnings.filterwarnings('ignore')

rock_data = pd.read_csv("../input/emg-4/0.csv", header=None )
scissors_data = pd.read_csv("../input/emg-4/1.csv", header=None )
paper_data = pd.read_csv("../input/emg-4/2.csv", header=None )
ok_data = pd.read_csv("../input/emg-4/3.csv", header=None )
data = pd.concat([rock_data, scissors_data, paper_data, ok_data], axis = 0)


X = data.drop(data.columns[-1],axis=1)
y = data[data.columns[-1]]

#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train.values.reshape(-1, 1)).todense()
y_test = one_hot.transform(y_test.values.reshape(-1, 1)).todense()

many_colors = [];
for i in range(200):
    many_colors.append((min(0.8,random.random()),min(0.8,random.random()),min(0.8,random.random())))

In [ ]:
print("SHAPE:",X.shape)

## Part 1: K-Means

In [ ]:
from sklearn.cluster import KMeans
wcss = []
x_vals = []
for i in range(1, 20):
    t = time.time()
    x_vals.append(int(i**1.8))
    kmeans = KMeans(n_clusters = int(i**1.8), init = 'k-means++', random_state = 42)
    kmeans.fit(X_train)
    # inertia method returns wcss for that model
    wcss.append(kmeans.inertia_)
    print(int(i**1.8), "intertia:",kmeans.inertia_,"\ttime:",time.time()-t)

In [ ]:
plt.figure(figsize=(8,5))
sns.lineplot(x_vals, wcss,marker='o',color='red')
plt.title('Gesture Recognition: Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
np_X_train = X_train.to_numpy()[0:300]


clusts = 45
kmeans = KMeans(n_clusters = clusts, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(np_X_train)

for i in range(1,2):
    # Visualising the clusters
    plt.figure(figsize=(12,8))
    
    x_variable = 0
    for k in range(clusts):
        sns.scatterplot(np_X_train[(y_kmeans == k), x_variable], np_X_train[y_kmeans == k, i+1], color= many_colors[k],s=50)#label = 'Cluster '+str(k+1)
    #sns.scatterplot(np_X_train[y_kmeans == 1, x_variable], np_X_train[y_kmeans == 1, i+1], color = 'blue', label = 'Cluster 2',s=50)
    #sns.scatterplot(np_X_train[y_kmeans == 2, x_variable], np_X_train[y_kmeans == 2, i+1], color = 'green', label = 'Cluster 3',s=50)
    #sns.scatterplot(np_X_train[y_kmeans == 3, x_variable], np_X_train[y_kmeans == 3, i+1], color = 'purple', label = 'Cluster 4',s=50)
    #sns.scatterplot(np_X_train[y_kmeans == 4, x_variable], np_X_train[y_kmeans == 4, i+1], color = 'orange', label = 'Cluster 5',s=50)
    sns.scatterplot(kmeans.cluster_centers_[:, x_variable], kmeans.cluster_centers_[:, i+1], color = 'red', alpha=0.3,
                    label = 'Centroids',s=500,marker='o')
    
    plt.grid(False)
    plt.title('K-Means Clustering')
    plt.xlabel('Feature '+str(x_variable))
    plt.ylabel('Feature '+str(i))
    #plt.legend()
    plt.show()

## Expecting Maximization

In [ ]:
bics = [];
aics = [];
avgs = [];
x_vals = []

for i in range(1,20):
    this_x = int(i**1.4)
    print(this_x)
    mixture = GaussianMixture(n_components=this_x, init_params='kmeans')
    mixture.fit(X_train);
    bics.append(mixture.bic(X_train));
    aics.append(mixture.aic(X_train));
    avgs.append((mixture.bic(X_train)+mixture.aic(X_train))/2)
    x_vals.append(this_x);

plt.figure(figsize=(8,5))
sns.lineplot(x_vals, bics,marker='o',color='red', label="BIC")
sns.lineplot(x_vals, aics,marker='o',color='blue', label="AIC")
#sns.lineplot(x_vals, avgs,marker='o',color='green', label="Avg")
plt.title('Gesture Recognition: Expecting Maximization')
plt.xlabel('Number of clusters')
plt.ylabel('Criterion Value')
plt.legend()
plt.show()

In [ ]:
np_X_train = X_train.to_numpy()[0:300]

colors = ['blue','green','purple','orange','m','y','forestgreen','darkorange','darkblue','indigo','brown','steelblue','chocolate','olivedrab']

clusts = 6
kmeans = GaussianMixture(n_components=clusts, init_params='kmeans')
y_kmeans = kmeans.fit_predict(np_X_train);

for i in range(1,2):#3,4
    # Visualising the clusters
    plt.figure(figsize=(12,8))
    
    x_variable = 0
    for k in range(clusts):
        sns.scatterplot(np_X_train[(y_kmeans == k), x_variable], np_X_train[y_kmeans == k, i+1], color= colors[k], label = 'Cluster '+str(k+1),s=50)
    #sns.scatterplot(np_X_train[y_kmeans == 1, x_variable], np_X_train[y_kmeans == 1, i+1], color = 'blue', label = 'Cluster 2',s=50)
    #sns.scatterplot(np_X_train[y_kmeans == 2, x_variable], np_X_train[y_kmeans == 2, i+1], color = 'green', label = 'Cluster 3',s=50)
    #sns.scatterplot(np_X_train[y_kmeans == 3, x_variable], np_X_train[y_kmeans == 3, i+1], color = 'purple', label = 'Cluster 4',s=50)
    #sns.scatterplot(np_X_train[y_kmeans == 4, x_variable], np_X_train[y_kmeans == 4, i+1], color = 'orange', label = 'Cluster 5',s=50)
    sns.scatterplot(kmeans.means_[:, x_variable], kmeans.means_[:, i+1], color = 'red', alpha=0.3,
                    label = 'Centroids',s=500,marker='o')
    
    plt.grid(False)
    plt.title('Expecting Maximization')
    plt.xlabel('Feature 0')
    plt.ylabel('Feature 1')
    plt.legend()
    plt.show()

## PCA INITIAL

In [ ]:
losses = [];
for i in range(1,64):
    pca = PCA(n_components=i)
    X_train_ica = pca.fit_transform(X_train)
    X_projected = pca.inverse_transform(X_train_ica)

    loss = np.mean((X_train - X_projected) ** 2)
    #print(i,"LOSS:",np.mean(list(loss)))
    losses.append(np.mean(list(loss)))
    
plt.plot(range(1,64),losses,color='g')
plt.rcParams["figure.figsize"] = (10,6)
plt.xlabel('New Features')
plt.ylabel('Eigenvalues')
plt.title("PCA Eigenvalues by Number of New Features")
plt.legend()
plt.show()

In [ ]:
losses = []

for n in range(1,64):
    total_kurtosis = [];
    for i in range(1,n):
        ica = FastICA(n_components=i)
        X_train_ica = ica.fit_transform(X_train)
        dt = tree.DecisionTreeClassifier(random_state=0)

        this_x = np.array(X_train)[:,i]
        new_x = []
        for j in range(len(this_x)):
            new_x.append([this_x[j]])
        #print("TX:",new_x[0:5])
        dt.fit(new_x,y_train)
        preds = dt.predict(new_x)
        #print(i,"Kurtosis:",kurtosis(preds))
        total_kurtosis.append(kurtosis(preds))
    print(n,"Avg kurtosis:", np.mean(total_kurtosis),end="\t")
    if n > 1: losses.append(np.mean(total_kurtosis))
plt.plot(range(2,64),losses,marker='s',color='g')
plt.rcParams["figure.figsize"] = (10,6)
plt.xlabel('New Features')
plt.ylabel('Kurtosis')
plt.title("ICA Kurtosis by Number of New Features")
plt.show()

In [ ]:
total_losses = []

best_loss = 999999;
best_index = 0;
for i in range(1,64):
    losses = []
    for j in range(100):
        pca = GaussianRandomProjection(n_components=i)
        pca.fit(X_train)
        data_reduced = np.dot(X_train, pca.components_.T) # transform
        X_projected = np.dot(data_reduced, pca.components_)# inverse_transform
        loss = np.mean((X_train - X_projected) ** 2)
        ##print(i,"LOSS:",np.mean(list(loss)))
        losses.append(np.mean(list(loss)))
    if i % 5 == 0: print(i,"LOSS:",np.mean(losses));
    total_losses.append(np.mean(losses))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_index = i;
print("Best loss, index:",best_loss, best_index)

plt.plot(range(1,64),total_losses,color='g')
plt.rcParams["figure.figsize"] = (10,6)
plt.xlabel('New Features')
plt.ylabel('Reconstruction Error')
plt.title("Randomized Projections Reconstruction Error by Number of New Features")
plt.show()

#Uses reconstructed Error
#After running 100 times each, 12 is the best number of components.
#Use 4 because of elbow method

In [ ]:
losses = [];
for n in range(1,64):
    total_kurtosis = [];
    for i in range(1,n):
        ica = FactorAnalysis(n_components=i)
        X_train_ica = ica.fit_transform(X_train)
        dt = tree.DecisionTreeClassifier(random_state=0)

        this_x = np.array(X_train)[:,i]
        new_x = []
        for j in range(len(this_x)):
            new_x.append([this_x[j]])
        #print("TX:",new_x[0:5])
        dt.fit(new_x,y_train)
        preds = dt.predict(new_x)
        #print(i,"Kurtosis:",kurtosis(preds))
        total_kurtosis.append(kurtosis(preds))
    print(n,"Avg kurtosis:", np.mean(total_kurtosis),end="\t")
    if i > 1: losses.append(np.mean(total_kurtosis))
#Best kurtosis of -1.091 with 6 components

In [ ]:
plt.plot(range(2,56),losses,marker='s',color='g')
plt.rcParams["figure.figsize"] = (10,6)
plt.xlabel('New Features')
plt.ylabel('Kurtosis')
plt.title("Factor Analysis Kurtosis by Number of New Features")
plt.show()

## Draw K-Means with Reduction:

In [ ]:
from sklearn.cluster import KMeans
wcss_list = [[],[],[],[]]
x_vals = []
X_train_list = [PCA(n_components=30).fit_transform(X_train),
                FastICA(n_components=21).fit_transform(X_train),
                GaussianRandomProjection(n_components=10).fit_transform(X_train),
                FactorAnalysis(n_components=21).fit_transform(X_train)]

for i in range(1, 20):
    t = time.time()
    x_vals.append(int(i**1.5))
    
    for j in range(4):
        kmeans = KMeans(n_clusters = int(i**1.5), init = 'k-means++', random_state = 42)
        kmeans.fit(X_train_list[j])
        wcss_list[j].append(kmeans.inertia_)
    print(i,"t:",time.time()-t,end="\t")
   
#Plot the charts
plt.figure(figsize=(8,5))
##plt.yscale("log")
sns.lineplot(x_vals, wcss_list[0],marker='o',color='red',label="PCA")
sns.lineplot(x_vals, wcss_list[1],marker='o',color='blue',label="ICA")
sns.lineplot(x_vals, wcss_list[2],marker='o',color='orange',label="RP")
sns.lineplot(x_vals, wcss_list[3],marker='o',color='purple',label="FA")
plt.title('Gesture Recognition: Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
print("ICA:",wcss_list[1])
print("X vals",x_vals)

In [ ]:
reduction_models = [PCA(n_components=2), FastICA(n_components=2), GaussianRandomProjection(n_components=2), FactorAnalysis(n_components=2)]
random.shuffle(many_colors)

clusters = [4,4,4,4]
f_list = [[0,1],[0,1],[0,1],[0,1]]
#bests: 0,1   3,4  0,1  0,1
titles = ["PCA", "ICA", "Random Projection", "Factor Analysis"]

np_X_train = X_train.to_numpy()[0:300]

for i in range(len(reduction_models)):
    model = reduction_models[i];
    model.fit_transform(np_X_train);
    model_X_train = model.transform(np_X_train);
    model_X_test = model.transform(X_test)
    
    kmeans = KMeans(n_clusters = clusters[i], init = 'k-means++')
    y_kmeans = kmeans.fit_predict(model_X_train)
    #print("Y MEANS:",y_kmeans)

    plt.figure(figsize=(12,8))
    for k in range(clusters[i]):#try with 3 and for, and 7 and 4.
        sns.scatterplot(model_X_train[(y_kmeans == k), f_list[i][0]], model_X_train[y_kmeans == k, f_list[i][1]], color= many_colors[k],s=50)#, label = 'Cluster '+str(k+1)
        
    sns.scatterplot(kmeans.cluster_centers_[:, f_list[i][0]], kmeans.cluster_centers_[:, f_list[i][1]], color = 'red', alpha=0.3,
                    label = 'Centroids',s=500,marker='o')
    plt.grid(False)
    plt.title('K-Means Clustering with '+titles[i])
    plt.xlabel('Reduced Feature '+str(f_list[i][0]))
    plt.ylabel('Reduced Feature '+str(f_list[i][1]))
    #plt.legend()
    plt.show()
    

## NOW USE EM:

In [ ]:
X_train_list = [PCA(n_components=30).fit_transform(X_train),
                FastICA(n_components=21).fit_transform(X_train),
                GaussianRandomProjection(n_components=10).fit_transform(X_train),
                FactorAnalysis(n_components=21).fit_transform(X_train)]
model_titles = ["PCA","ICA","RP", "FA"]
aic_colors = ['red','orange','brown','gray']
bic_colors = ['blue', 'green', 'purple', 'black']

plt.figure(figsize=(10,6))

for a in range(4):
    bics = [];
    aics = [];
    avgs = [];
    x_vals = []

    for i in range(1,20):
        this_x = int(i**1.4)
        print(i, this_x,end="\t")
        mixture = GaussianMixture(n_components=this_x, init_params='kmeans')
        mixture.fit(X_train_list[a]);
        bics.append(mixture.bic(X_train_list[a]));
        aics.append(mixture.aic(X_train_list[a]));
        avgs.append((mixture.bic(X_train_list[a])+mixture.aic(X_train_list[a]))/2)
        x_vals.append(this_x);
    sns.lineplot(x_vals, bics,marker='o',color=aic_colors[a], label=model_titles[a] + " BIC")
    sns.lineplot(x_vals, aics,marker='o',color=bic_colors[a], label=model_titles[a] + " AIC")


#sns.lineplot(x_vals, avgs,marker='o',color='green', label="Avg")
plt.title('Gesture Recognition: Expecting Maximization')
plt.xlabel('Number of clusters')
plt.ylabel('Criterion Value')
plt.legend()
plt.show()
for i in range(10):
    print(int(i**1.4),end=",")
#print()

In [ ]:
reduction_models = [PCA(n_components=2), FastICA(n_components=2), GaussianRandomProjection(n_components=2), FactorAnalysis(n_components=2)]
random.shuffle(many_colors)

clusters = [4,4,4,4]
f_list = [[0,1],[0,1],[0,1],[0,1]]
#bests: 0,1   3,4  0,1  0,1
titles = ["PCA", "ICA", "Random Projection", "Factor Analysis"]

np_X_train = X_train.to_numpy()[0:300]

for i in range(len(reduction_models)):
    model = reduction_models[i];
    model.fit_transform(np_X_train);
    model_X_train = model.transform(np_X_train);
    model_X_test = model.transform(X_test)
    
    kmeans = GaussianMixture(n_components=clusters[i], init_params='kmeans')
    y_kmeans = kmeans.fit_predict(model_X_train)
    #print("Y MEANS:",y_kmeans)

    plt.figure(figsize=(12,8))
    for k in range(clusters[i]):#try with 3 and for, and 7 and 4.
        sns.scatterplot(model_X_train[(y_kmeans == k), f_list[i][0]], model_X_train[y_kmeans == k, f_list[i][1]], color= many_colors[k],s=50)#, label = 'Cluster '+str(k+1)
        
    #sns.scatterplot(kmeans.cluster_centers_[:, f_list[i][0]], kmeans.cluster_centers_[:, f_list[i][1]], color = 'red', alpha=0.3,
    #                label = 'Centroids',s=500,marker='o')
    plt.grid(False)
    plt.title('Expecting Maximization Clustering with '+titles[i])
    plt.xlabel('Reduced Feature '+str(f_list[i][0]))
    plt.ylabel('Reduced Feature '+str(f_list[i][1]))
    #plt.legend()
    plt.show()
    